# Osnovne obrade prirodnog jezika 

## (NLP, eng. *Natural Language Processing*)

[ćeliju izvršavate klikom na nju i zatim Shift+Return, ili samo klikom na ikonicu "Run cell"]

U ovom ćemo notebooku pokazati kako se računaju osnovne NLP mjere uz pomoć Pythona. Prvi dio se sastoji od ucitavanja oglednog teksta. U ovom slučaju to Moby Dick koji je referenciran kao **mobydick.txt**. Prva linija otvara file i stavlja ga u memoriju na koju pokazuje varijabla **file_opened**, druga linija čita iz memorije i vraća string koji sadrži tekst i sprema je u varijablu **text_content**. Da bi se to dogodilo izvršite ćeliju ispod:

In [ ]:
file_opened = open('mobydick.txt', 'r')
text_content = file_opened.read().lower()

> Kao dodatnu vježbu, modificirajte gornji kod da ispiše cijeli tekst.

Prvo možemo ispisati broj riječi u fileu:

In [ ]:
list_words = text_content.split()
print(len(list_words))

Možemo sada maknuti sve rijeci duljine 3 znaka ili manje (uočimo da će broj riječi pasti za 40%):

In [ ]:
list_words = [x for x in text_content.split() if len(x)>3]
print(len(list_words))

Sada ispišemo broj jedinstvenih riječi (svaku različitu brojimo samo jednom):

In [ ]:
set_words = set(text_content.split())
print(len(set_words))

Sada potražimo koliko se puta javlja riječ "whale":

In [ ]:
n_whale = len([x for x in list_words if x=="whale"])
print(n_whale)

Sada pogledajmo koliko ima različitih rijeći koje su dulje od 3 slova:

In [ ]:
n_more_than_3 = len([x for x in set_words if len(x)>3])
print(n_more_than_3)

> Kao zadatak, promjenite gornji kod da vam pokaže koliko ima riječi duljih od 10 znakova.

Ispisimo sve riječi dulje od 20 znakova:

In [ ]:
more_than_20 = [x for x in set_words if len(x)>20]
print(more_than_20)

Kao što vidimo ovdje ima svačega, ba bismo mogli ovo izbaciti iz našeg skupa riječi:

In [ ]:
less_than_20_set = [x for x in set_words if len(x)<20]

Napravimo listu svih bigrama u Moby Dicku i ispišimo prvih 10:

In [ ]:
from nltk import bigrams
bg = bigrams(list_words)
bigrami = list(bg)
print(bigrami[:10])

> Probajte modificirati gornji kod da ispiše sve bigrame

Sljedeće ćemo pronaći kolokacije. 

Reimplementirat ćemo algoritam u čistom Pythonu. 

Krenimo od definicije zajedničke vjerojatnosti. Ako je vjerojatnost događaja $A$ označena kao $P(A)$ i vjerojatnost događaja $B$ kao $P(B)$, vjerojatnost njihovog zajedničke pojave $P(A\cap B)$ je $P(A)\cdot P(B)$.

Vjerojatnost događaja $A$ se računa pobrojavanjem. Ako je A opisan kao "Pojava riječi "whale" u Moby Dicku", tada je njena vjerojatnost opisana kao kvocijent broja pojave te riječi (532 puta) i sveukupnog broja riječi (215139 riječi)

> Iskoristite Python za izračun ovog kvocijenta:

In [ ]:
# ovdje napisite svoj kod za izracun kvocijenta

Izračunajmo vjerojatnost pojave riječi "white":

In [ ]:
p_whale = 532/215139 

n_white = len([x for x in list_words if x=="white"])

p_white = n_white/215139

p_whiteAndWhale = p_white * p_whale

print(p_whiteAndWhale)

Sada nadimo broj pojava bigrama "white whale" i podijelimo ju s brojem svih bigrama:

In [ ]:
dulj_bg = len(bigrami) #uocite da je ovo uvijek duljine n-1 ako je n broj rijeci
whiteWhale_lista = [x for x in bigrami if x == ('white','whale')]
p_whiteWhale = len(whiteWhale_lista)/dulj_bg
print(p_whiteWhale - p_whiteAndWhale)

Čime dobijemo razliku između vjerojatnosti da dvije riječi slučajno dođu zajedno i da su one kolokacija. Ovisno o pragu koji postavimo, neki bigram može biti kolokacija ili ne. Standardan prag je 0.01%, što znači da je "white whale" ovdje kolokacija.

Sa pragom morate eksperimentirati jer će ponekad 0.01% biti previše a nekada premalo. To eksperimentiranje se radi tako da se definira funkcija koja će ovu vrijednost računati za svaki bigram, i onda vratiti sortiranu listu bigrama čija vrijednost prelazi neki prag.

To ćemo napraviti sljedeće (sve pokrećemo ispočetka ovdje, tako da je ova ćelija samostalni program):

## (samostalni kod)

In [ ]:
from nltk import bigrams
import time
start_time = time.time()
file_opened = open('mobydick.txt', 'r')#ovdje umjesto mobydick.txt stavite ime svog filea
text_content = file_opened.read().lower()
list_words = text_content.split()#[:5000]# obrisite samo prvu povisilicu da biste dobili prvih 5000 rijeci

br_rijeci = len(list_words)

bg = bigrams(list_words)
bigrami = list(bg)

def add_scores_to_bigrams(bigram_list, list_of_all_words):
    list_bigrams_new = []
    for i in bigram_list:
        word1 = i[0]
        word2 = i[1]
        
        n_word1 = len([x for x in list_words if x==word1])
        n_word2 = len([x for x in list_words if x==word2])
        p_word1 = n_word1 / len(list_of_all_words)
        p_word2 = n_word2 / len(list_of_all_words)
        p_word1andword2 = p_word1 * p_word2
        
        n_bigram = len([x for x in bigram_list if x == (word1,word2)])
        p_bigram = n_bigram/len(bigram_list)
        
        score = p_bigram - p_word1andword2

        trio = [score,[word1,word2]]
        #print(trio)
        if trio not in list_bigrams_new:
            list_bigrams_new.append(trio)
        #print(list_bigrams_new[:3])

    return sorted(list_bigrams_new,key=lambda x: x[0], reverse=True)


   
results = add_scores_to_bigrams(bigrami,list_words)#[:20]# obrisite prvu povisilicu da biste limitirali na 20 najznacajnih kolokacija 
print("--- Current runtime is %s seconds ---" % (time.time() - start_time))

for i in results:
    print(i)





## Bag of Words (BOW) kreator

Ovaj dio notebooka je samostalan i kod koji je dolje je tzv. Bag of Words kreator. Bag of Words je najjednostavniji model za obradu prirodnog jezika. 

Zamislimo da imamo neki standardni tekst podijeljen na neke fragmente. To može biti na primjer file s 2000 Facebook komentara. Tada svi oni zajedno predstavljanu cijeli *dokument*, a svaki komentar predstavlja jedan *fragment*. Naravno, Facebook komentari mogu sadržavati više od jedne rečenice, pa zato fragment i rečenica nisu jedno te isto, ali ako imamo drugačiji dokument oni to mogu biti. Fragmente definiramo kako želimo, a jedino je važno da su oni relativno mali komadi s obzirom na duljinu dokument. Idealno bi oni trebali tvoriti neke intuitivne cjeline koje imaju smisla (jedan FB komentar je jedna cjelina jer ju je napisao u istom trenutku isti autor), ali je izbor na pojedinom analitičaru da odredi što mu je smisleni fragment.

Zamislimo da svaki fragment dodatno ima oznaku sentimenta (ručno označenu) koja je N (negativni) ili P (pozitivni). Ovo tvori jedan jednostavan CSV koji ima retke oblika:

> TEXT,SENTIMENT

> "to je to:-)",P

> "to je bzvz",N

> "tvoj komentar je bzvz",N

> ...


Bag of Words (BOW) model će ovo isto prikazati na drugačiji način. Retci će isto kao i gore predstavljati iste fragmente (istim redom), ali imena stupaca će postati sve riječi koje se javljaju u dokumentu (odnosno koje se javljaju u osnovnom stupcu "TEXT"). 

U određenom smislu, BOW uzima CSV koji ima jedan stupac teksta i možda još neke stupce i iz njega stvara drugi CSV koji prepiše ostale stupce, a supac TEXT zamijeni s N stupaca koji svaki ima za naziv neku riječ iz dokumenta.

U retku se zapisuje 0 kao vrijednost stupca "xyz" ako taj fragment nema riječ "xyz" u sebi. Ako ima i pojavljuje se jednom, zapiše se 1, ako se pojavljuje dva puta zapisuje se 2, itd.

Gornji CSV bi kada bi se pretvorio u BOW izgledao ovako:

> bzvz,je,komentar,to,tvoj,SENTIMENT

> 0,1,0,2,0,P

> 1,1,0,1,0,N

> 1,1,1,0,1,P


Što je konačni izgled CSV-a koji se dobije u outputu. 

Donja ćelija ce stvoriti iz CSV-a s retcima oblika (TEXT,OZNAKA) novi CSV oblika (riječ1,riječ2,rijec3,..., OZNAKA). Uz notebook imate i testni file koji se zove "test.csv" pa možete pogledati u Notepad++ kako on izgleda. Isto tako mora izgledati i Vaš file da ga skripta uspije obraditi i pretvoriti u BOW. Vaš file mora biti u istom direktoriju kao i ovaj notebook da bi stvar radila. Ovaj kod će iz njega napraviti output file u istom direktoriju gdje je i notebook koji će se zvati OUTPUT_FILE.csv.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

in_file = "test.csv"
out_file = "OUTPUT_FILE.csv"


df0 = pd.read_csv(in_file, encoding="cp1250")

lista =[]
for i in df0.iterrows():
    lista.append(i[1][0])

    

vectorizer = CountVectorizer(min_df=1)
podatci = vectorizer.fit_transform(lista)

izvorni_tekst_za_join = np.asarray(lista)
row_labels = pd.DataFrame(izvorni_tekst_za_join, columns=['label'])
imena_za_cols = np.asarray(vectorizer.get_feature_names())

df1 = pd.DataFrame(podatci.toarray(), columns=imena_za_cols)

df3 = pd.concat([df1.drop_duplicates(), df0.drop_duplicates()], axis=1)

df3 = df3.drop(["TEXT"],axis=1)

df3.to_csv(out_file, encoding="cp1250", index=False) # utf8

Možemo ispisati ulazni CSV da vidimo kako izgleda:

In [ ]:
df0

A sada ispišemo izlazni CSV (BOW + oznake sentimenta):

In [ ]:
df3